In [1]:
import PIL
from PIL import Image
import numpy as np
import os 
import random
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

from keras.wrappers.scikit_learn import KerasClassifier
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
from keras import regularizers

Using TensorFlow backend.
/home/jan/anaconda3/envs/tf-gpu-clone/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jan/anaconda3/envs/tf-gpu-clone/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jan/anaconda3/envs/tf-gpu-clone/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jan/a

In [2]:
dirs = ['/media/jan/Volume_big/datasets/Second_try/Grid_size_874_binary/3layer/no_porosity',
        '/media/jan/Volume_big/datasets/Second_try/Grid_size_874_binary/3layer/porosity']

length = 2000
image_paths = []
labels = []
data = []

In [3]:
height = 128
width = 128
depth = 3
inputShape = (height, width, depth)

In [5]:
for d in dirs: 
    cur_paths = []
    for path in os.listdir(d):
        full_path = os.path.join(d,path)
        cur_paths.append(full_path)
        
    random.seed(42)
    random.shuffle(cur_paths)
    cur_paths = cur_paths[:length]
    
    image_paths.append(cur_paths)        

image_paths = [val for sublist in image_paths for val in sublist]
random.seed(42)
random.shuffle(image_paths)

######
for path in image_paths:
    label = path.split(os.path.sep)[-2]
    labels.append(label)
    
    array = np.load(path)
    img = Image.fromarray(array)
    img = img.resize((height, width), PIL.Image.LANCZOS)
    array_resized_flattened = np.array(img)
    data.append(array_resized_flattened)
    
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
trainY = np.hstack((trainY, 1- trainY))

testY = lb.transform(testY)
testY = np.hstack((testY, 1- testY))

In [6]:
def create_model():
    
    model = Sequential()
    model.add(Conv2D(3,(6,6), padding="same", input_shape = inputShape))   #, kernel_regularizer=regularizers.l2(0.01)))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis = -1))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(6, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=-1))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(32))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    
    model.add(Dense(2))
    model.add(Activation("sigmoid"))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model 

In [7]:
trainY.shape

(3000, 2)

In [8]:
trainY[1]

array([1, 0])

In [9]:
model = KerasClassifier(build_fn=create_model)

In [22]:
batch_size = [8, 16, 32]
epochs = [20, 40, 60 ]
param_grid = dict(batch_size=batch_size, epochs=epochs)

In [23]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)

In [24]:
grid_result = grid.fit(trainX, trainY)

Epoch 1/20
2000/2000 [==============================] - 8s 4ms/step - loss: 0.6749 - acc: 0.6238
Epoch 2/20
2000/2000 [==============================] - 3s 2ms/step - loss: 0.6019 - acc: 0.6840
Epoch 3/20
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5869 - acc: 0.6827
Epoch 4/20
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5698 - acc: 0.6995
Epoch 5/20
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5709 - acc: 0.6927
Epoch 6/20
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5518 - acc: 0.7022
Epoch 7/20
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5459 - acc: 0.7248
Epoch 8/20
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5461 - acc: 0.7175
Epoch 9/20
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5400 - acc: 0.7243
Epoch 10/20
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5378 - acc: 0.7167
Epoch 11/20
2000/2000 [======

2000/2000 [==============================] - 3s 2ms/step - loss: 0.5456 - acc: 0.7230
Epoch 24/40
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5313 - acc: 0.7340
Epoch 25/40
2000/2000 [==============================] - 3s 2ms/step - loss: 0.4996 - acc: 0.7508
Epoch 26/40
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5221 - acc: 0.7410
Epoch 27/40
2000/2000 [==============================] - 3s 2ms/step - loss: 0.4929 - acc: 0.7610
Epoch 28/40
2000/2000 [==============================] - 3s 2ms/step - loss: 0.4865 - acc: 0.7572
Epoch 29/40
2000/2000 [==============================] - 3s 2ms/step - loss: 0.4768 - acc: 0.7680
Epoch 30/40
2000/2000 [==============================] - 3s 2ms/step - loss: 0.4651 - acc: 0.7765
Epoch 31/40
2000/2000 [==============================] - 3s 2ms/step - loss: 0.4665 - acc: 0.7750
Epoch 32/40
2000/2000 [==============================] - 3s 2ms/step - loss: 0.4483 - acc: 0.7848
Epoch 33/40
2000/2000 [=========

2000/2000 [==============================] - 3s 2ms/step - loss: 0.5204 - acc: 0.7300
Epoch 26/40
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5229 - acc: 0.7332
Epoch 27/40
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5105 - acc: 0.7492
Epoch 28/40
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5158 - acc: 0.7233
Epoch 29/40
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5118 - acc: 0.7385
Epoch 30/40
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5275 - acc: 0.7208
Epoch 31/40
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5165 - acc: 0.7345
Epoch 32/40
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5029 - acc: 0.7415
Epoch 33/40
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5051 - acc: 0.7418
Epoch 34/40
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5200 - acc: 0.7220
Epoch 35/40
2000/2000 [=========

2000/2000 [==============================] - 3s 2ms/step - loss: 0.5561 - acc: 0.7082
Epoch 8/60
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5439 - acc: 0.7177
Epoch 9/60
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5315 - acc: 0.7190
Epoch 10/60
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5348 - acc: 0.7210
Epoch 11/60
2000/2000 [==============================] - 4s 2ms/step - loss: 0.5093 - acc: 0.7370
Epoch 12/60
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5384 - acc: 0.7243
Epoch 13/60
2000/2000 [==============================] - 4s 2ms/step - loss: 0.5301 - acc: 0.7265
Epoch 14/60
2000/2000 [==============================] - 4s 2ms/step - loss: 0.5200 - acc: 0.7300
Epoch 15/60
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5105 - acc: 0.7452
Epoch 16/60
2000/2000 [==============================] - 3s 2ms/step - loss: 0.4870 - acc: 0.7598
Epoch 17/60
2000/2000 [===========

2000/2000 [==============================] - 3s 2ms/step - loss: 0.2919 - acc: 0.8750
Epoch 31/60
2000/2000 [==============================] - 4s 2ms/step - loss: 0.3886 - acc: 0.8280
Epoch 32/60
2000/2000 [==============================] - 4s 2ms/step - loss: 0.5121 - acc: 0.7568
Epoch 33/60
2000/2000 [==============================] - 3s 2ms/step - loss: 0.4323 - acc: 0.8022
Epoch 34/60
2000/2000 [==============================] - 4s 2ms/step - loss: 0.3763 - acc: 0.8320
Epoch 35/60
2000/2000 [==============================] - 4s 2ms/step - loss: 0.2952 - acc: 0.8765
Epoch 36/60
2000/2000 [==============================] - 4s 2ms/step - loss: 0.2771 - acc: 0.8900
Epoch 37/60
2000/2000 [==============================] - 4s 2ms/step - loss: 0.2480 - acc: 0.8978
Epoch 38/60
2000/2000 [==============================] - 4s 2ms/step - loss: 0.2627 - acc: 0.8930
Epoch 39/60
2000/2000 [==============================] - 4s 2ms/step - loss: 0.2528 - acc: 0.9005
Epoch 40/60
2000/2000 [=========

...

1000/1000 [==============================] - 4s 4ms/step
Epoch 1/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.6874 - acc: 0.6238
Epoch 2/20
2000/2000 [==============================] - 1s 664us/step - loss: 0.5851 - acc: 0.6882
Epoch 3/20
2000/2000 [==============================] - 1s 655us/step - loss: 0.5649 - acc: 0.6873
Epoch 4/20
2000/2000 [==============================] - 1s 660us/step - loss: 0.5565 - acc: 0.6947
Epoch 5/20
2000/2000 [==============================] - 1s 651us/step - loss: 0.5430 - acc: 0.7053
Epoch 6/20
2000/2000 [==============================] - 1s 646us/step - loss: 0.5345 - acc: 0.7062
Epoch 7/20
2000/2000 [==============================] - 1s 659us/step - loss: 0.5198 - acc: 0.7160
Epoch 8/20
2000/2000 [==============================] - 1s 664us/step - loss: 0.5161 - acc: 0.7205
Epoch 9/20
2000/2000 [==============================] - 1s 677us/step - loss: 0.4966 - acc: 0.7478
Epoch 10/20
2000/2000 [==============================

1000/1000 [==============================] - 5s 5ms/step
Epoch 1/40
2000/2000 [==============================] - 12s 6ms/step - loss: 0.6503 - acc: 0.6577
Epoch 2/40
2000/2000 [==============================] - 1s 688us/step - loss: 0.5757 - acc: 0.6975
Epoch 3/40
2000/2000 [==============================] - 1s 672us/step - loss: 0.5526 - acc: 0.7040
Epoch 4/40
2000/2000 [==============================] - 1s 675us/step - loss: 0.5394 - acc: 0.7065
Epoch 5/40
2000/2000 [==============================] - 1s 685us/step - loss: 0.5239 - acc: 0.7258
Epoch 6/40
2000/2000 [==============================] - 1s 690us/step - loss: 0.4888 - acc: 0.7488
Epoch 7/40
2000/2000 [==============================] - 1s 693us/step - loss: 0.4671 - acc: 0.7690
Epoch 8/40
2000/2000 [==============================] - 1s 671us/step - loss: 0.4360 - acc: 0.7873
Epoch 9/40
2000/2000 [==============================] - 1s 671us/step - loss: 0.3914 - acc: 0.8192
Epoch 10/40
2000/2000 [==============================

1000/1000 [==============================] - 5s 5ms/step
Epoch 1/60
2000/2000 [==============================] - 14s 7ms/step - loss: 0.6615 - acc: 0.6628
Epoch 2/60
2000/2000 [==============================] - 1s 697us/step - loss: 0.5687 - acc: 0.7015
Epoch 3/60
2000/2000 [==============================] - 1s 688us/step - loss: 0.5487 - acc: 0.7155
Epoch 4/60
2000/2000 [==============================] - 1s 685us/step - loss: 0.5287 - acc: 0.7283
Epoch 5/60
2000/2000 [==============================] - 1s 667us/step - loss: 0.5041 - acc: 0.7445
Epoch 6/60
2000/2000 [==============================] - 1s 667us/step - loss: 0.4633 - acc: 0.7818
Epoch 7/60
2000/2000 [==============================] - 1s 664us/step - loss: 0.4358 - acc: 0.7905
Epoch 8/60
2000/2000 [==============================] - 1s 666us/step - loss: 0.3930 - acc: 0.8190
Epoch 9/60
2000/2000 [==============================] - 1s 674us/step - loss: 0.3508 - acc: 0.8520
Epoch 10/60
2000/2000 [==============================

Epoch 22/60
2000/2000 [==============================] - 1s 672us/step - loss: 0.1140 - acc: 0.9620
Epoch 23/60
2000/2000 [==============================] - 1s 660us/step - loss: 0.1126 - acc: 0.9643
Epoch 24/60
2000/2000 [==============================] - 1s 673us/step - loss: 0.0976 - acc: 0.9730
Epoch 25/60
2000/2000 [==============================] - 1s 671us/step - loss: 0.0938 - acc: 0.9647
Epoch 26/60
2000/2000 [==============================] - 1s 685us/step - loss: 0.0813 - acc: 0.9718
Epoch 27/60
2000/2000 [==============================] - 1s 685us/step - loss: 0.0710 - acc: 0.9780
Epoch 28/60
2000/2000 [==============================] - 1s 682us/step - loss: 0.0829 - acc: 0.9725
Epoch 29/60
2000/2000 [==============================] - 1s 681us/step - loss: 0.0796 - acc: 0.9765
Epoch 30/60
2000/2000 [==============================] - 1s 693us/step - loss: 0.0709 - acc: 0.9770
Epoch 31/60
2000/2000 [==============================] - 1s 687us/step - loss: 0.0719 - acc: 0.9758


2000/2000 [==============================] - 1s 694us/step - loss: 0.0454 - acc: 0.9862
Epoch 44/60
2000/2000 [==============================] - 1s 702us/step - loss: 0.0447 - acc: 0.9843
Epoch 45/60
2000/2000 [==============================] - 1s 684us/step - loss: 0.0445 - acc: 0.9870
Epoch 46/60
2000/2000 [==============================] - 1s 696us/step - loss: 0.0593 - acc: 0.9780
Epoch 47/60
2000/2000 [==============================] - 1s 684us/step - loss: 0.0580 - acc: 0.9792
Epoch 48/60
2000/2000 [==============================] - 1s 677us/step - loss: 0.0450 - acc: 0.9850
Epoch 49/60
2000/2000 [==============================] - 1s 689us/step - loss: 0.0421 - acc: 0.9873
Epoch 50/60
2000/2000 [==============================] - 1s 686us/step - loss: 0.0452 - acc: 0.9852
Epoch 51/60
2000/2000 [==============================] - 1s 684us/step - loss: 0.0553 - acc: 0.9813
Epoch 52/60
2000/2000 [==============================] - 1s 677us/step - loss: 0.0355 - acc: 0.9880
Epoch 53/60


In [25]:
grid_result.cv_results_

{'mean_fit_time': array([ 73.68156854, 143.68014566, 217.118457  ,  47.42368134,
         89.58552599, 134.55436039,  37.21461654,  66.33197959,
         95.801958  ]),
 'std_fit_time': array([0.21188654, 1.14867643, 1.70199721, 0.7021431 , 0.20612979,
        1.95143775, 0.63712292, 1.22583816, 0.36364875]),
 'mean_score_time': array([2.58037845, 2.87928422, 3.28858956, 3.28682788, 3.70697006,
        4.11455655, 4.35195009, 5.08571426, 5.49253813]),
 'std_score_time': array([0.06545984, 0.09177454, 0.23222692, 0.08418371, 0.06928528,
        0.15753874, 0.10290539, 0.35226705, 0.16243076]),
 'param_batch_size': masked_array(data=[8, 8, 8, 16, 16, 16, 32, 32, 32],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_epochs': masked_array(data=[20, 40, 60, 20, 40, 60, 20, 40, 60],
              mask=[False, False, False, False, False, False, False, False,
                    F

In [26]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.705167 using {'batch_size': 8, 'epochs': 20}
0.705167 (0.012795) with: {'batch_size': 8, 'epochs': 20}
0.703833 (0.007192) with: {'batch_size': 8, 'epochs': 40}
0.692833 (0.008270) with: {'batch_size': 8, 'epochs': 60}
0.686167 (0.010881) with: {'batch_size': 16, 'epochs': 20}
0.684500 (0.010985) with: {'batch_size': 16, 'epochs': 40}
0.700333 (0.006196) with: {'batch_size': 16, 'epochs': 60}
0.696667 (0.012250) with: {'batch_size': 32, 'epochs': 20}
0.687000 (0.010206) with: {'batch_size': 32, 'epochs': 40}
0.699833 (0.010781) with: {'batch_size': 32, 'epochs': 60}
